In [1]:
from result_saver import SaverProvider
from Scratch import metadata_loader
from Scratch import find_closest_calib_jobs

import numpy as np
from tqdm import tqdm
from time import sleep
from datetime import datetime
import json
import os

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
import cpp_soft_info as csi

def decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name):
    provider = SaverProvider()

    state = "X" if XBASIS else "Z"
    state += LOGICAL
    print(f"state = {state}")


    # Load the metadata
    while True:
        try:
            md = metadata_loader(True, True)
            break
        except:
            sleep(5)
    md = md[md["job_status"] == "JobStatus.DONE"]
    md = md[md["code"] == "RepetitionCodeCircuit"]
    md = md[md["descr"] == 'subset RepCodes']
    md = md.dropna(subset=["rounds"])
    md = md[md["meas_level"] == 1]
    md['rounds'] = md['rounds'].astype(int)
    md['distance'] = md['distance'].astype(int)

    md = md[md["backend_name"] == DEVICE]
    md = md[md["logical"] == LOGICAL]
    md = md[md["xbasis"] == XBASIS]
    md = md[md["rounds"] == ROUNDS]

    print("shape:", md.shape)

    DOUBLE_MSMT = False # to get also 03-16

    jobs_by_calibration_date = {}
    for index, row in md.iterrows():
        job_id = row['job_id']

        while True:
            try:
                _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
                break
            except:
                sleep(5)

        if calib_creation_date not in jobs_by_calibration_date.keys():
            jobs_by_calibration_date[calib_creation_date] = [job_id]
        else:
            jobs_by_calibration_date[calib_creation_date].append(job_id)

    print(jobs_by_calibration_date)
    print()
    print(f"num of calibrations: {len(jobs_by_calibration_date)}")
    print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")


    d = md["distance"].values[0]

    distances = np.arange(7, d+1, 4)
    distances = distances[::-1]

    
    HANDLE_OUTLIERS = True

    rel_error = 1
    _RESETS = False

    # KDE BANDWIDTHS
    lin = [0.6, 1.2, 1]
    num_points = 20
    # lin = [0.1, 0.7, 1]
    # num_points = 7
    bandwidths = np.linspace(lin[0], lin[1], lin[2])


    for calib_date in jobs_by_calibration_date.keys():
        # get the noise dict of that date
        noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

        # get the KDE of that date

        while True:
            try:
                all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=False)
                break
            except:
                sleep(5)

        kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

        while True:
            try:
                all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                                other_date=calib_date, post_process=True,
                                                                                double_msmt=True)
                break
            except:
                sleep(5)
        kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

        # Get the mean msmt errors
        p_soft_mean = 0
        p_hard_mean = 0
        for key, value in msmt_err_dict_PS.items():
            p_soft_mean += value['p_soft']
            p_hard_mean += value['p_hard']
        p_soft_mean /= len(msmt_err_dict_PS)
        p_hard_mean /= len(msmt_err_dict_PS)
        print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)

        for i in range(127):
            print(f"qubit {i}: {kde_dict[i].bestBandwidth}", end=' ')
        print("\nPS", end=' ')
        for i in range(127):
            print(f"qubit {i}: {kde_dict_PS[i].bestBandwidth}", end=' ')

        # Retrieve the memories
        memories = []
        for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
            d = md[md["job_id"] == job_id]["distance"].values[0] 
            T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

            # Get the job
            job = provider.retrieve_job(job_id)
            memory = job.result().get_memory()
            memories.append(memory)

        # Stack the memories vertically
        big_memory = np.vstack(memories)

        # Get the layout of the last job (same as previous)
        layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
        link_qubits = list(layout_des['link_qubit'].values())
        code_qubits = list(layout_des['code_qubit'].values())

        # Get the pSoft and countMat matrices
        big_layout = link_qubits + code_qubits
        inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

        print(f"Starting to get pSoft and countMat at {datetime.now()}")
        pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
                                                    handleOutliers = HANDLE_OUTLIERS)
        
        print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
        pSoft_PS, countMat_PS = csi.iqConvertor(big_memory, inverted_q_map, kde_dict_PS, rel_error, -1,
                                                        handleOutliers = HANDLE_OUTLIERS)
        
        print(f"Starting to get pSoftG at {datetime.now()}")
        countMatG, pSoftG = gaussianIQConvertor(big_memory, inverted_q_map, gmm_dict)

        assert pSoft.shape[0] == pSoft_PS.shape[0] == pSoftG.shape[0], "Different number of shots for each decoding method"
        NB_SHOTS = pSoft.shape[0]

        # Subsample decoding
        for D_NEW in tqdm(distances):
            subsets = generate_subsets_with_center(d, D_NEW)

            err_s_G_mean = []
            err_s_G_indiv = []
            err_h_G_indiv = []
            err_h_G_mean = []
            err_s_K_mean = []
            err_s_K_indiv = []
            err_h_K_indiv = []
            err_h_K_mean = []
            err_s_KPS_mean = []
            err_s_KPS_indiv = []
            err_h_KPS_indiv = []
            err_h_KPS_mean = []
            err_h_G_mean_mean = []
            err_h_K_mean_mean = []
            err_h_KPS_mean_mean = []
            err_h_G_mean_mean_50 = []
            err_h_K_mean_mean_50 = []
            err_h_KPS_mean_mean_50 = []
            for subset in (subsets):             
                cols_to_keep = get_cols_to_keep(subset, T, d)

                # Get the subset of pSoft and countMat
                pSoft_sub = pSoft[:, cols_to_keep]
                pSoft_PS_sub = pSoft_PS[:, cols_to_keep]
                pSoftG_sub = pSoftG[:, cols_to_keep]
                countMat_sub = countMat[:, cols_to_keep]
                countMat_PS_sub = countMat_PS[:, cols_to_keep]
                countMatG_sub = countMatG[:, cols_to_keep]

                # Means for hard decoding
                pSoft_mean = np.mean(pSoft_sub)
                pSoft_PS_mean = np.mean(pSoft_PS_sub)
                pSoftG_mean = np.mean(pSoftG_sub)

                # Get the layout
                layout = get_subsample_layout(subset, link_qubits, code_qubits)

                # Get the noise avgs
                avgs = get_avgs_from_dict(noise_dict, layout)
                noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
                readout = avgs["readout"]
                noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]
                #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

                # Stim models
                subsampling = (D_NEW != d)

                code_indiv_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                                subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                                msmt_err_dict=msmt_err_dict_PS)
                model_indiv_for_soft = code_indiv_for_soft.circuits[LOGICAL].detector_error_model()

                code_indiv_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                                subsampling=subsampling, no_fin_soft=False, layout=layout,
                                                                msmt_err_dict=msmt_err_dict_PS)
                model_indiv_for_hard = code_indiv_for_hard.circuits[LOGICAL].detector_error_model()

                code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                                subsampling=subsampling, no_fin_soft=True, layout=None,
                                                                msmt_err_dict=None)
                model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

                code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                                subsampling=subsampling, no_fin_soft=False, layout=None,
                                                                msmt_err_dict=None)
                model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()
                

                # Decoding

                ### Gaussian
                res_s_G_mean = csi.decodeConvertorAll(model_mean_for_soft, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
                res_s_G_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMatG_sub, pSoftG_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_h_G_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMatG_sub, pSoftG_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                res_h_G_mean = csi.decodeConvertorAll(model_mean_for_hard, countMatG_sub, pSoftG_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                
                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoftG_mean
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                                subsampling=subsampling, no_fin_soft=False, layout=None,
                                                                msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_G_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMatG_sub, pSoftG_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)

                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoftG_mean*1.5
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                                subsampling=subsampling, no_fin_soft=False, layout=None,
                                                                msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_G_mean_mean_50 = csi.decodeConvertorAll(model_mean_mean, countMatG_sub, pSoftG_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                
                ### KDE
                res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_s_K_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_h_K_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                
                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoft_mean
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_K_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_sub, pSoft_sub, T,
                                                            int(LOGICAL), _RESETS, decode_hard=True)
                
                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoft_mean*1.5
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_K_mean_mean_50 = csi.decodeConvertorAll(model_mean_mean, countMat_sub, pSoft_sub, T,
                                                            int(LOGICAL), _RESETS, decode_hard=True)
                
                
                ### KDE PS
                res_s_KPS_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_s_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_h_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                res_h_KPS_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                
                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoft_PS_mean
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_KPS_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_PS_sub, pSoft_PS_sub, T,
                                                            int(LOGICAL), _RESETS, decode_hard=True)
                
                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoft_PS_mean*1.5
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_KPS_mean_mean_50 = csi.decodeConvertorAll(model_mean_mean, countMat_PS_sub, pSoft_PS_sub, T,
                                                            int(LOGICAL), _RESETS, decode_hard=True)
                
                # Append the results
                err_s_G_mean.append(res_s_G_mean.num_errors)
                err_s_G_indiv.append(res_s_G_indiv.num_errors)
                err_h_G_indiv.append(res_h_G_indiv.num_errors)
                err_h_G_mean.append(res_h_G_mean.num_errors)
                err_s_K_mean.append(res_s_K_mean.num_errors)
                err_s_K_indiv.append(res_s_K_indiv.num_errors)
                err_h_K_indiv.append(res_h_K_indiv.num_errors)
                err_h_K_mean.append(res_h_K_mean.num_errors)
                err_s_KPS_mean.append(res_s_KPS_mean.num_errors)
                err_s_KPS_indiv.append(res_s_KPS_indiv.num_errors)
                err_h_KPS_indiv.append(res_h_KPS_indiv.num_errors)
                err_h_KPS_mean.append(res_h_KPS_mean.num_errors)

                err_h_G_mean_mean.append(res_h_G_mean_mean.num_errors)
                err_h_K_mean_mean.append(res_h_K_mean_mean.num_errors)
                err_h_KPS_mean_mean.append(res_h_KPS_mean_mean.num_errors)

                err_h_G_mean_mean_50.append(res_h_G_mean_mean_50.num_errors)
                err_h_K_mean_mean_50.append(res_h_K_mean_mean_50.num_errors)
                err_h_KPS_mean_mean_50.append(res_h_KPS_mean_mean_50.num_errors)


            # Save the results
            json_Gaussian = {
                "decoding": "gaussian",
                "d_new": str(D_NEW),
                "error_list_dict": {
                    "soft_mean": err_s_G_mean,
                    "soft_indiv": err_s_G_indiv,
                    "hard_indiv": err_h_G_indiv,
                    "hard_mean": err_h_G_mean,
                    "hard_mean_mean": err_h_G_mean_mean,
                    "hard_mean_mean_50": err_h_G_mean_mean_50,
                },
                "error_mean_dict": {
                    "soft_mean": np.mean(err_s_G_mean),
                    "soft_indiv": np.mean(err_s_G_indiv),
                    "hard_indiv": np.mean(err_h_G_indiv),
                    "hard_mean": np.mean(err_h_G_mean),
                    "hard_mean_mean": np.mean(err_h_G_mean_mean),
                    "hard_mean_mean_50": np.mean(err_h_G_mean_mean_50),
                },      
                "additional_info": {
                    "job_ids": jobs_by_calibration_date[calib_date],
                    "calib_date": str(calib_date),
                    "noise_list": noise_list,
                    "calib_readout": readout,
                    "p_soft_mean": p_soft_mean,
                    "p_hard_mean": p_hard_mean,
                    "pSoftG_mean": pSoftG_mean,
                    "pSoftG_mean*1.5": pSoftG_mean*1.5,
                },    
            }

            json_KDE = {
                "decoding": "kde",
                "d_new": str(D_NEW),
                "error_list_dict": {
                    "soft_mean": err_s_K_mean,
                    "soft_indiv": err_s_K_indiv,
                    "hard_indiv": err_h_K_indiv,
                    "hard_mean": err_h_K_mean,
                    "hard_mean_mean": err_h_K_mean_mean,
                    "hard_mean_mean_50": err_h_K_mean_mean_50,
                },
                "error_mean_dict": {
                    "soft_mean": np.mean(err_s_K_mean),
                    "soft_indiv": np.mean(err_s_K_indiv),
                    "hard_indiv": np.mean(err_h_K_indiv),
                    "hard_mean": np.mean(err_h_K_mean),
                    "hard_mean_mean": np.mean(err_h_K_mean_mean),
                    "hard_mean_mean_50": np.mean(err_h_K_mean_mean_50),
                },   
                "additional_info": {
                    "rel_error": rel_error,
                    "bandwidth_linspace": lin,
                    "num_points_bandwidths": num_points,
                    "pSoft_mean": pSoft_mean,
                    "pSoft_mean*1.5": pSoft_mean*1.5,
                },        
            }

            json_KDE_PS = {
                "decoding": "kde_PS",
                "d_new": str(D_NEW),
                "error_list_dict": {
                    "soft_mean": err_s_KPS_mean,
                    "soft_indiv": err_s_KPS_indiv,
                    "hard_indiv": err_h_KPS_indiv,
                    "hard_mean": err_h_KPS_mean,
                    "hard_mean_mean": err_h_KPS_mean_mean,
                    "hard_mean_mean_50": err_h_KPS_mean_mean_50,
                },
                "error_mean_dict": {
                    "soft_mean": np.mean(err_s_KPS_mean),
                    "soft_indiv": np.mean(err_s_KPS_indiv),
                    "hard_indiv": np.mean(err_h_KPS_indiv),
                    "hard_mean": np.mean(err_h_KPS_mean),
                    "hard_mean_mean": np.mean(err_h_KPS_mean_mean),
                    "hard_mean_mean_50": np.mean(err_h_KPS_mean_mean_50),
                },   
                "additional_info": {
                    "pSoft_PS_mean": pSoft_PS_mean,
                    "pSoft_PS_mean*1.5": pSoft_PS_mean*1.5,
                },
            }

            if not os.path.exists(file_name):
                data = {}
            else:
                with open(file_name, "r") as f:
                    data = json.load(f)

            if job_id not in data.keys():
                data[job_id] = [{'totshots': NB_SHOTS}, json_Gaussian, json_KDE, json_KDE_PS]
            else:
                data[job_id].append(json_Gaussian)
                data[job_id].append(json_KDE)
                data[job_id].append(json_KDE_PS)
            
            with open(file_name, "w") as f:
                json.dump(data, f, indent=4)

In [2]:
logicals = ['0', '1']
xbasis = [True, False]
rounds = [50]
devices = ["ibm_torino"]

for DEVICE in devices:
    for LOGICAL in logicals:
        for XBASIS in xbasis:
            for ROUNDS in rounds:
                state = "X" if XBASIS else "Z"
                state += LOGICAL

                file_name  = f'./results/results_night2/{DEVICE}_{state}_{ROUNDS}.json'

                decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name)

state = X0
shape: (40, 18)
{datetime.datetime(2024, 3, 24, 15, 4, 50, 495000, tzinfo=datetime.timezone.utc): ['cr04230ktf3g008854s0', 'cr0421gk5z700081s380', 'cr041zz8gdp0008fxc5g', 'cr041yfdvs8g008j7rm0', 'cr041wzczq6g0081jwx0', 'cr041vfktf3g008854qg', 'cr041szdvs8g008j7rk0', 'cr041rfczq6g0081jwwg', 'cr041pyczq6g0081jww0', 'cr041ne8gdp0008fxc4g', 'cr041kps9z7g008dqszg', 'cr041j6ktf3g008854q0'], datetime.datetime(2024, 3, 24, 15, 2, 38, 518000, tzinfo=datetime.timezone.utc): ['cr041gek5z700081s36g', 'cr041exk5z700081s360', 'cr041d5k5z700081s35g', 'cr041bxdvs8g008j7rj0', 'cr041a5dvs8g008j7rhg', 'cr0418nktf3g008854p0', 'cr04174dvs8g008j7rh0', 'cr0415m8gdp0008fxc30'], datetime.datetime(2024, 3, 16, 15, 10, 31, 675000, tzinfo=datetime.timezone.utc): ['cqtvcrk4x0mg008acny0', 'cqtvcq288ev000813ang', 'cqtvcnt88ev000813an0', 'cqtvcma88ev000813amg', 'cqtvck2pkcdg008e3qa0', 'cqtvchj9nfw0008ht26g', 'cqtvcga36d60008j01eg', 'cqtvcf188ev000813akg', 'cqtvcdh4x0mg008acnwg', 'cqtvcc188ev000813ak0', 'cq

Retrieving jobs of 2024-03-24 15:04:50.495000+00:00 calibration: 100%|██████████| 12/12 [00:30<00:00,  2.54s/it]


Starting to get pSoft and countMat at 2024-04-02 23:07:09.547600
Starting to get pSoft_PS and countMat_PS at 2024-04-02 23:11:41.386938
Starting to get pSoftG at 2024-04-02 23:14:41.043809


100%|██████████| 13/13 [31:59<00:00, 147.65s/it]


23:46:55 Warning: Could not get two gate error of ECR due to 'Could not find the desired property for ecr', taking CX instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 67
23:46:55 Warning: Could not get two gate error of CX due to 'Could not find the desired property for cx', taking 0.5 instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 71
Found jobs for backend ibm_torino with closest execution date 2024-03-24 15:40:13.116247+00:00.
Found jobs for backend ibm_torino with closest execution date 2024-03-24 15:40:40.928003+00:00.
{0: {'p_hard': 0.09536476690829288, 'p_soft': 0.012169980714238212}, 1: {'p_hard': 0.011205692624858682, 'p_soft': 0.0026268537607235486}, 2: {'p_hard': 0.021280840593203433, 'p_soft': 0.017024672474562744}, 3: {'p_hard': 0.01389904901243599, 'p_soft': 0.0009642880893795305}

...

100%|██████████| 13/13 [47:17<00:00, 218.30s/it] 


09:09:00 Warning: Z0 decoding. Negative T2 error -0.00 % for qubit 12, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
09:09:04 Warning: Could not get two gate error of ECR due to 'Could not find the desired property for ecr', taking CX instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 67
09:09:04 Warning: Could not get two gate error of CX due to 'Could not find the desired property for cx', taking 0.5 instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 71
Found jobs for backend ibm_torino with closest execution date 2024-03-16 14:56:22.522112+00:00.
Found jobs for backend ibm_torino with closest execution date 2024-03-23 08:50:56.582662+00:00.
{0: {'p_hard': 0.08844849371550176, 'p_soft': 0.01

Retrieving jobs of 2024-03-16 15:06:05.732000+00:00 calibration: 100%|██████████| 9/9 [00:23<00:00,  2.63s/it]


Starting to get pSoft and countMat at 2024-04-03 09:10:48.904775
Starting to get pSoft_PS and countMat_PS at 2024-04-03 09:14:33.213898
Starting to get pSoftG at 2024-04-03 09:17:00.958931


100%|██████████| 13/13 [40:10<00:00, 185.43s/it] 


In [4]:
logicals = ['0', '1']
xbasis = [True, False]
rounds = [75]
devices = ["ibm_sherbrooke"]

for DEVICE in devices:
    for LOGICAL in logicals:
        for XBASIS in xbasis:
            for ROUNDS in rounds:
                state = "X" if XBASIS else "Z"
                state += LOGICAL

                file_name  = f'./results/results_night2/{DEVICE}_{state}_{ROUNDS}.json'

                decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name)

state = X0
state = X0
shape: (40, 18)
{datetime.datetime(2024, 3, 23, 15, 19, 27, 45000, tzinfo=datetime.timezone.utc): ['cqzf5wq8gdp0008fvx0g', 'cqzf5sfktf3g00883rxg', 'cqzf5p6ktf3g00883rwg', 'cqzf5jys9z7g008dpdm0', 'cqzf5fndvs8g008j6cfg', 'cqzf5cdczq6g0081hgr0', 'cqzf595ktf3g00883rw0', 'cqzf56cs9z7g008dpdkg', 'cqzf52wk5z70008j0az0', 'cqzf4zbczq6g0081hgpg', 'cqzf4vvczq6g0081hgng'], datetime.datetime(2024, 3, 23, 15, 14, 57, 515000, tzinfo=datetime.timezone.utc): ['cqzf4rb8gdp0008fvwy0', 'cqzf4mtk5z70008j0awg', 'cqzf4hjs9z7g008dpdhg', 'cqzf4e9ktf3g00883rs0', 'cqzf4b18gdp0008fvwx0', 'cqzf481dvs8g008j6cd0', 'cqzf45gk5z70008j0avg', 'cqzf428ktf3g00883rr0', 'cqzf3yfk5z70008j0av0'], datetime.datetime(2024, 3, 16, 18, 28, 35, 391000, tzinfo=datetime.timezone.utc): ['cqty9jetxzj0008y0g40', 'cqty9fn4x0mg008ad0yg', 'cqty9cx4x0mg008ad0y0', 'cqty9adtxzj0008y0g30', 'cqty97wtxzj0008y0g2g', 'cqty9549nfw0008ht80g', 'cqty92m88ev000813geg', 'cqty8zkpkcdg008e3xwg', 'cqty8x34x0mg008ad0wg', 'cqty8tk88ev000

Retrieving jobs of 2024-03-23 15:19:27.045000+00:00 calibration: 100%|██████████| 11/11 [00:28<00:00,  2.56s/it]


Starting to get pSoft and countMat at 2024-04-02 06:46:57.860884
Starting to get pSoft_PS and countMat_PS at 2024-04-02 06:51:04.024830
Starting to get pSoftG at 2024-04-02 06:53:38.011354


100%|██████████| 12/12 [26:23<00:00, 131.93s/it]


07:20:09 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
07:20:09 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
07:20:09 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 14:56:07.669856+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 14:56:37.347221+00:00.
{0: {'p_hard': 0.005270510541021082, 'p_soft': 0.000190500381000762}, 1: {'p_hard': 0.006254762509525018, 'p_soft': 0.004254

...

Starting to get pSoft and countMat at 2024-04-02 08:27:20.583842
Starting to get pSoft_PS and countMat_PS at 2024-04-02 08:30:43.458327
Starting to get pSoftG at 2024-04-02 08:32:58.632206


100%|██████████| 12/12 [25:05<00:00, 125.43s/it]


state = Z0
state = Z0
shape: (40, 18)
{datetime.datetime(2024, 3, 23, 15, 10, 49, 958000, tzinfo=datetime.timezone.utc): ['cqzf1vz8gdp0008fvwq0', 'cqzf1sqk5z70008j0afg', 'cqzf1py8gdp0008fvwpg', 'cqzf1m6k5z70008j0af0', 'cqzf1hp8gdp0008fvwp0', 'cqzf1exs9z7g008dpd90', 'cqzf1cds9z7g008dpd8g', 'cqzf1a5s9z7g008dpd80', 'cqzf17ck5z70008j0adg', 'cqzf14wk5z70008j0ac0', 'cqzf12m8gdp0008fvwn0', 'cqzf104s9z7g008dpd60', 'cqzf0xk8gdp0008fvwmg'], datetime.datetime(2024, 3, 23, 15, 7, 4, 748000, tzinfo=datetime.timezone.utc): ['cqzf0tvs9z7g008dpd50', 'cqzf0rbczq6g0081hgfg', 'cqzf0njczq6g0081hgf0', 'cqzf0htczq6g0081hge0', 'cqzf0ehktf3g00883rgg', 'cqzf0bhktf3g00883rfg', 'cqzf08hktf3g00883rf0'], datetime.datetime(2024, 3, 16, 18, 19, 33, 528000, tzinfo=datetime.timezone.utc): ['cqty56c4x0mg008ad0b0', 'cqty53w4x0mg008ad0a0', 'cqty51436d60008j05n0', 'cqty4ykpkcdg008e3xg0', 'cqty4d1txzj0008y0fs0', 'cqty4ah36d60008j05mg', 'cqty47rtxzj0008y0frg', 'cqty4584x0mg008ad080', 'cqty42gtxzj0008y0fr0', 'cqty3zq88ev0008

Retrieving jobs of 2024-03-23 15:10:49.958000+00:00 calibration: 100%|██████████| 13/13 [00:34<00:00,  2.64s/it]


Starting to get pSoft and countMat at 2024-04-02 09:00:16.721585
